# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [ ]:
%load_ext dotenv
%dotenv 
import os
import sys
sys.path.append(os.getenv('SRC_DIR'))
import dask.dataframe as dd
import pandas as pd
import numpy as np

from glob import glob

In [5]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


In [6]:
adult_dt.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')

# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [13]:
X = adult_dt.drop('income', axis= 1)
Y = adult_dt['income']
adult_dt.shape, X.shape , Y.shape

((32561, 15), (32561, 14), (32561,))

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

adult_dt.shape, X_train.shape, Y_train.shape, X_test.shape, Y_test.shape



((32561, 15), (22792, 14), (22792,), (9769, 14), (9769,))

## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)*

random state of the splitting function is 42 
It is very much helpful to assign a particular random state to make the troubleshooting and assessment easier. Random state helps in reducing the randomness and to ensure that re-run of the code still generates the same samples of the data. 



# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [23]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score, log_loss, cohen_kappa_score, f1_score


In [38]:
#Getting to know datatypes 
print(X.dtypes)

#Numerical Variables  
X_numerical = X.select_dtypes(exclude=['object', 'category', 'bool', 'datetime', 'timedelta' ] )

#Categorical Variables  
X_categorical = X.select_dtypes(include=['object', 'category'])

# Validations  
X_numerical.dtypes, X_categorical.dtypes



age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
dtype: object


(age               int64
 fnlwgt            int64
 education-num     int64
 capital-gain      int64
 capital-loss      int64
 hours-per-week    int64
 dtype: object,
 workclass         object
 education         object
 marital-status    object
 occupation        object
 relationship      object
 race              object
 sex               object
 native-country    object
 dtype: object)

In [41]:
# Applying custom transformations based on datatypes 

# Numerical Transformations 
num_trans = Pipeline(
    [ 
        ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
        ('Scaler', RobustScaler())
    ]
)


# Categorical Transformations 
cat_trans = Pipeline(
    [ 
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))
    ]
)

# Designing Custom ColumnTransformer 
preprocessor = ColumnTransformer(
    transformers=[
        ('num_transforms', num_trans, X_numerical.columns ),
        ('cat_transforms', cat_trans, X_categorical.columns ),
    ], remainder='drop'
)


# visualizing the preprocessor columntransformer
preprocessor

ColumnTransformer(transformers=[('num_transforms',
                                 Pipeline(steps=[('imputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('Scaler', RobustScaler())]),
                                 Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object')),
                                ('cat_transforms',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'))]),
                                 Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object'))])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [43]:
from sklearn.ensemble import RandomForestClassifier

pipe = Pipeline(
    [
        ('preprocessing', preprocessor), 
        ('clasifier', RandomForestClassifier(random_state=42))
    ]

)

#showing the pipeline without fit 
pipe 

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('Scaler',
                                                                   RobustScaler())]),
                                                  Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object')),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object'))])),
                ('clasifier', RandomForestClassifier(random_state=42))])

In [44]:
#Fit the data in the model and pipeline 
pipe.fit(X_train, Y_train)




Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('Scaler',
                                                                   RobustScaler())]),
                                                  Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object')),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object'))])),
                ('clasifier', RandomForestClassifier(random_state=42))])

In [45]:
# Getting the predictions

Y_pred_train = pipe.predict(X_train)
Y_pred_test = pipe.predict(X_test)

Y_pred_test


array([0, 0, 1, ..., 0, 1, 1])

In [46]:
# Calculating Prediction Probabilities 

Y_proba_train = pipe.predict_proba(X_train)
Y_proba_test = pipe.predict_proba(X_test)

Y_proba_test

array([[1.  , 0.  ],
       [0.57, 0.43],
       [0.31, 0.69],
       ...,
       [1.  , 0.  ],
       [0.47, 0.53],
       [0.02, 0.98]])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [58]:
from sklearn.model_selection import cross_validate

scoring = ['accuracy', 'balanced_accuracy',  'f1', 'precision', 'recall', 'roc_auc', 'neg_log_loss', 'neg_brier_score']
d3_dict = cross_validate(pipe, X, Y, cv=5, scoring = scoring, return_train_score = True)

c:\Users\engr_\.conda\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\engr_\.conda\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [59]:
d3_dict

{'fit_time': array([18.36956811, 18.54956269, 17.70136547, 17.70603371, 18.03100991]),
 'score_time': array([0.33169985, 0.291363  , 0.28774762, 0.31039166, 0.30888724]),
 'test_accuracy': array([0.85398434, 0.84874079, 0.85273342, 0.85795455, 0.85764742]),
 'train_accuracy': array([1.        , 0.99996161, 0.99996161, 1.        , 0.99996161]),
 'test_balanced_accuracy': array([0.77025226, 0.76712445, 0.77889918, 0.7779828 , 0.78017572]),
 'train_balanced_accuracy': array([1.        , 0.99997472, 0.99997472, 1.        , 0.99997472]),
 'test_f1': array([0.66759874, 0.6599931 , 0.67546531, 0.67893093, 0.68089501]),
 'train_f1': array([1.       , 0.9999203, 0.9999203, 1.       , 0.9999203]),
 'test_precision': array([0.73916409, 0.71933785, 0.71953857, 0.7448591 , 0.73971578]),
 'train_precision': array([1.        , 0.99984061, 0.99984061, 1.        , 0.99984061]),
 'test_recall': array([0.60866794, 0.60969388, 0.63647959, 0.62372449, 0.6307398 ]),
 'train_recall': array([1., 1., 1., 1., 1

Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [60]:
results_folded = pd.DataFrame(d3_dict).sort_values(by = ['test_neg_log_loss'])
results_folded

,fit_time,score_time,test_accuracy,train_accuracy,test_balanced_accuracy,train_balanced_accuracy,test_f1,train_f1,test_precision,train_precision,test_recall,train_recall,test_roc_auc,train_roc_auc,test_neg_log_loss,train_neg_log_loss,test_neg_brier_score,train_neg_brier_score
1,18.549563,0.291363,0.848741,0.999962,0.767124,0.999975,0.659993,0.99992,0.719338,0.999841,0.609694,1.0,0.896894,1.0,-0.404425,-0.079798,-0.105489,-0.014107
0,18.369568,0.331700,0.853984,1.000000,0.770252,1.000000,0.667599,1.00000,0.739164,1.000000,0.608668,1.0,0.902940,1.0,-0.362914,-0.079765,-0.102833,-0.014136
3,17.706034,0.310392,0.857955,1.000000,0.777983,1.000000,0.678931,1.00000,0.744859,1.000000,0.623724,1.0,0.901815,1.0,-0.358425,-0.080946,-0.102709,-0.014271
4,18.031010,0.308887,0.857647,0.999962,0.780176,0.999975,0.680895,0.99992,0.739716,0.999841,0.630740,1.0,0.907448,1.0,-0.350558,-0.080935,-0.100151,-0.014319
2,17.701365,0.287748,0.852733,0.999962,0.778899,0.999975,0.675465,0.99992,0.719539,0.999841,0.636480,1.0,0.906473,1.0,-0.348129,-0.080182,-0.102007,-0.014143


Calculate the mean of each metric. 

In [61]:
results_folded.mean()

fit_time                   18.071508
score_time                  0.306018
test_accuracy               0.854212
train_accuracy              0.999977
test_balanced_accuracy      0.774887
train_balanced_accuracy     0.999985
test_f1                     0.672577
train_f1                    0.999952
test_precision              0.732523
train_precision             0.999904
test_recall                 0.621861
train_recall                1.000000
test_roc_auc                0.903114
train_roc_auc               1.000000
test_neg_log_loss          -0.364890
train_neg_log_loss         -0.080325
test_neg_brier_score       -0.102638
train_neg_brier_score      -0.014195
dtype: float64

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [62]:
from sklearn.metrics import balanced_accuracy_score


def evaluate_model(clf, X_train, Y_train, X_test, Y_test):
    clf.fit(X_train, Y_train)
    Y_pred_train = clf.predict(X_train)
    Y_pred_test = clf.predict(X_test)
    Y_proba_train = clf.predict_proba(X_train)
    Y_proba_test = clf.predict_proba(X_test)
    performance_metrics = {
        'log_loss_train': log_loss(Y_train, Y_proba_train),
        'log_loss_test': log_loss(Y_test, Y_proba_test),
        'cohen_kappa_train': cohen_kappa_score(Y_train, Y_pred_train),
        'cohen_kappa_test': cohen_kappa_score(Y_test, Y_pred_test),
        'f1_score_train': f1_score(Y_train, Y_pred_train),
        'f1_score_test': f1_score(Y_test, Y_pred_test),
        'accuracy_score_train': accuracy_score(Y_train, Y_pred_train),
        'accuracy_score_test': accuracy_score(Y_test, Y_pred_test),
        'balanced_accuracy_score_train': balanced_accuracy_score(Y_train, Y_pred_train),
        'balanced_accuracy_score_test': balanced_accuracy_score(Y_test, Y_pred_test),
        'roc_auc_score_train': Y_proba_train[:,1],
        'roc_auc_score_test': Y_proba_test[:,1]
    }
    return performance_metrics

In [63]:
evaluate_model(pipe, X_train, Y_train, X_test, Y_test)

{'log_loss_train': 0.08064531163200583,
 'log_loss_test': 0.38733674303408167,
 'cohen_kappa_train': 1.0,
 'cohen_kappa_test': 0.5838055939538161,
 'f1_score_train': 1.0,
 'f1_score_test': 0.6753126447429365,
 'accuracy_score_train': 1.0,
 'accuracy_score_test': 0.8564847988535162,
 'balanced_accuracy_score_train': 1.0,
 'balanced_accuracy_score_test': 0.7784191753807199,
 'roc_auc_score_train': array([0.08, 0.97, 0.06, ..., 0.  , 0.15, 0.08]),
 'roc_auc_score_test': array([0.  , 0.43, 0.69, ..., 0.  , 0.53, 0.98])}

# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

In [ ]:
Immediate recoding of target variable income was needed based on the requirements. 
The problem here is mentioned as "Predict whether income exceeds $50K/yr based on census data". 
In this particular requirement, we just need to predict whether income is >50k or not hence response variable need to be recoded as binary. 
Doing this recoding early will ensure consistency and also help avoiding any data type issues later in the code. 


## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Evaluation of model pipeline |Model pipeline was evaluated correctly.|Model pipeline was not evaluated correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.